# 3 - Mapas

Vamos também gerar alguns mapas como parte de nossa análise de dados. Afinal, temos 96 gráficos, e precisamos extrair as principais informações dos dados. Os mapas que faremos serão:

- Atendimento (matrícula) absoluto por tempo.
- Atendimento (matrícula) relativo (relação à população) por tempo.
- Fila (demanda) absoluto por tempo.
- Fila (demanda) relativo por tempo.
- Variação entre junho de 2016 e dezembro de 2017 do atendimento absoluto.
- Variação entre junho de 2016 e dezembro de 2017 do atendimento relativo.

Primeiro, pegaremos os dados e faremos a função que gera os gráficos.

In [16]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import imageio
from pylab import rcParams
from matplotlib.pyplot import figure

rcParams['figure.figsize'] = 9, 12
rcParams['figure.dpi'] = 200

dados_educ_pop = pd.read_csv('dados/dados_educ_pop.csv', sep = ',')
distritos = gpd.read_file('dados/LAYER_DISTRITO/DEINFO_DISTRITO.shp')

for i in distritos.index:
    nome = distritos.at[i, 'NOME_DIST'].split(' ')
    if 'JD' in nome:
        nome[0] = 'JARDIM'
    if 'CID' in nome:
        nome[0] = 'CIDADE'
    distritos.at[i, 'NOME_DIST'] = ' '.join(nome)

def geraMapa(periodo, tema, atributo, colormap='inferno'):
    dados = dados_educ_pop[dados_educ_pop['MES_ANO_REF'] == periodo]
    dados = dados[['DISTRITO', atributo]]

    for i in distritos.index:
        recorte = dados[dados['DISTRITO'] == distritos.at[i, 'NOME_DIST']]
        distritos.at[i, atributo] = recorte[atributo].values[0]
            
    plt.ioff()
    ax = distritos.plot(column=atributo, cmap=colormap, legend=True)
    ax.set_title(periodo)
    ax.axis('off')
    plt.savefig('mapas/' + atributo + '_' + periodo + '.png', bbox_inches='tight')
    plt.close()
    
def geraGif(periodos, atributo):
    with imageio.get_writer('gifs/' + atributo + '.gif', mode='I', duration=0.2) as writer:
        for periodo in periodos:
            image = imageio.imread('mapas/' + atributo + '_' + periodo + '.png')
            writer.append_data(image)

## Mapas do atendimento absoluto por tempo

Para esses mapas, será gerado um imagem para cada período de referência (trimestre). Depois disso, será gerado um gif para ver melhor a evolução.

In [6]:
periodos = dados_educ_pop.MES_ANO_REF.unique()

for periodo in periodos:
    geraMapa(periodo=periodo, tema='Atendimento absoluto das creches', atributo='MAT_CRECHE', colormap='viridis')
    
geraGif(periodos, 'MAT_CRECHE')

IndexError: index 0 is out of bounds for axis 0 with size 0

## Mapa do atendimento relativo por tempo

Para esses mapas, será gerado um imagem para cada período de referência (trimestre). Depois disso, será gerado um gif para ver melhor a evolução.

In [14]:
dados_educ_pop['ATEND_REL'] = dados_educ_pop['MAT_CRECHE'] / dados_educ_pop['TOTAL']

for periodo in periodos:
    geraMapa(periodo=periodo, tema='Atendimento relativo à população', atributo='ATEND_REL', colormap="viridis")
    
geraGif(periodos, 'ATEND_REL')

## Mapa da fila absoluta por tempo

Para esses mapas, será gerado um imagem para cada período de referência (trimestre). Depois disso, será gerado um gif para ver melhor a evolução.

In [15]:
for periodo in periodos:
    geraMapa(periodo=periodo, tema='Demanda por vagas de creches', atributo='DEM_CRECHE')
    
geraGif(periodos, 'DEM_CRECHE')

## Mapa da fila relativa por tempo

Para esses mapas, será gerado um imagem para cada período de referência (trimestre). Depois disso, será gerado um gif para ver melhor a evolução.

In [16]:
dados_educ_pop['FILA_REL'] = dados_educ_pop['DEM_CRECHE'] / dados_educ_pop['TOTAL']

for periodo in periodos:
    geraMapa(periodo=periodo, tema='Demanda por vagas de creches', atributo='FILA_REL')
    
geraGif(periodos, 'FILA_REL')

## Mapa da evolução do número de matrículas

Esses mapas trarão a variação, do número de matrículas, de junho de 2006 a dezembro de 2017. O primeiro apresenta as vagas absolutas e o segundo, as relativas à população.

In [14]:
def geraMapaEvolucao(data, atributo, tema, dono, colormap="inferno"):
    data = data[['DISTRITO', atributo]]

    for i in data.index:
        recorte = data[data['DISTRITO'] == distritos.at[i, 'NOME_DIST']]
        distritos.at[i, atributo] = recorte[atributo].values[0]
            
    plt.ioff()
    ax = distritos.plot(column=atributo, cmap=colormap, legend=True)
    ax.set_title('')
    ax.axis('off')
    plt.savefig('mapas/' + atributo + '_' + dono + '.png', bbox_inches='tight')
    plt.close()

dados_matricula = dados_educ_pop[['DISTRITO', 'MAT_CRECHE', 'MES_ANO_REF']]
dados_jun_2006 = dados_matricula[dados_matricula['MES_ANO_REF'] == 'jun-06']
dados_jun_2006.columns = ['DISTRITO', 'MAT_JUN_06', 'MES_ANO_REF']
dados_dez_2017 = dados_matricula[dados_matricula['MES_ANO_REF'] == 'dez-17']
dados_dez_2017.columns = ['DISTRITO', 'MAT_DEZ_17', 'MES_ANO_REF']

dados_matricula = dados_jun_2006.merge(dados_dez_2017, on = ['DISTRITO'])
dados_matricula = dados_matricula[['DISTRITO', 'MAT_JUN_06', 'MAT_DEZ_17']]

dados_matricula['EV_ABS_NUM'] = dados_matricula['MAT_DEZ_17'] - dados_matricula['MAT_JUN_06']
dados_matricula['EV_ABS_PC'] = dados_matricula['MAT_DEZ_17'] / dados_matricula['MAT_JUN_06']
geraMapaEvolucao(dados_matricula, 'EV_ABS_NUM', dono='creche', tema="Evolução absoluta no número de matrículas dez-06 a dez-17", colormap="viridis")
geraMapaEvolucao(dados_matricula, 'EV_ABS_PC', dono='creche', tema="Evolução relativa no número de matrículas dez-06 a dez-17", colormap="viridis")

JOSE BONIFACIO
JARDIM SAO LUIS
ARTUR ALVIM
JAGUARA
VILA PRUDENTE
JAGUARE
JARAGUA
JARDIM HELENA
JARDIM PAULISTA
LAJEADO
LAPA
LIBERDADE
LIMAO
VILA MEDEIROS
JARDIM ANGELA
PARELHEIROS
VILA SONIA
AGUA RASA
ALTO DE PINHEIROS
ANHANGUERA
ARICANDUVA
BARRA FUNDA
BELA VISTA
BELEM
BOM RETIRO
BRAS
BRASILANDIA
BUTANTA
CACHOEIRINHA
CAMBUCI
CAMPO BELO
CAMPO GRANDE
CAMPO LIMPO
CANGAIBA
CAPAO REDONDO
CARRAO
CASA VERDE
CIDADE ADEMAR
CIDADE DUTRA
CIDADE LIDER
CIDADE TIRADENTES
CONSOLACAO
CURSINO
ERMELINO MATARAZZO
FREGUESIA DO O
GRAJAU
GUAIANASES
IGUATEMI
IPIRANGA
ITAIM BIBI
ITAIM PAULISTA
ITAQUERA
JABAQUARA
JACANA
MANDAQUI
MARSILAC
MOEMA
MOOCA
MORUMBI
PARI
PARQUE DO CARMO
PEDREIRA
PENHA
PERDIZES
PERUS
PINHEIROS
PIRITUBA
PONTE RASA
RAPOSO TAVARES
RIO PEQUENO
SACOMA
SANTANA
SANTO AMARO
SAO DOMINGOS
SAO LUCAS
SAO MATEUS
SAO MIGUEL
SAO RAFAEL
SAPOPEMBA
SAUDE
SE
SOCORRO
TATUAPE
TREMEMBE
TUCURUVI
VILA ANDRADE
VILA CURUCA
VILA FORMOSA
VILA GUILHERME
VILA JACUI
VILA LEOPOLDINA
VILA MARIA
VILA MARIANA
VILA MATILD

## Mapa da evolução da demanda

Esses mapas trará a variação, em número de crianças, de junho de 2006 a dezembro de 2017. O primeiro apresenta a demanda absoluta e o segundo, a relativa à população.

In [17]:
dados_demanda = dados_educ_pop[['DISTRITO', 'DEM_CRECHE', 'MES_ANO_REF']]
dados_jun_2006 = dados_demanda[dados_demanda['MES_ANO_REF'] == 'dez-06']
dados_jun_2006.columns = ['DISTRITO', 'DEM_JUN_06', 'MES_ANO_REF']
dados_dez_2017 = dados_demanda[dados_demanda['MES_ANO_REF'] == 'dez-17']
dados_dez_2017.columns = ['DISTRITO', 'DEM_DEZ_17', 'MES_ANO_REF']

dados_demanda = dados_jun_2006.merge(dados_dez_2017, on = ['DISTRITO'])
dados_demanda = dados_demanda[['DISTRITO', 'DEM_JUN_06', 'DEM_DEZ_17']]

dados_demanda['EV_ABS_NUM'] = dados_demanda['DEM_DEZ_17'] - dados_demanda['DEM_JUN_06']
dados_demanda['EV_ABS_PC'] = dados_demanda['DEM_DEZ_17'] / dados_demanda['DEM_JUN_06']
geraMapaEvolucao(dados_demanda, 'EV_ABS_NUM', dono='fila', tema="Evolução absoluta no tamanho da fila de dez-06 a dez-17")
geraMapaEvolucao(dados_demanda, 'EV_ABS_PC', dono='fila', tema="Evolução relativa no tamanho da fila de dez-06 a dez-17")

JOSE BONIFACIO
JARDIM SAO LUIS
ARTUR ALVIM
JAGUARA
VILA PRUDENTE
JAGUARE
JARAGUA
JARDIM HELENA
JARDIM PAULISTA
LAJEADO
LAPA
LIBERDADE
LIMAO
VILA MEDEIROS
JARDIM ANGELA
PARELHEIROS
VILA SONIA
AGUA RASA
ALTO DE PINHEIROS
ANHANGUERA
ARICANDUVA
BARRA FUNDA
BELA VISTA
BELEM
BOM RETIRO
BRAS
BRASILANDIA
BUTANTA
CACHOEIRINHA
CAMBUCI
CAMPO BELO
CAMPO GRANDE
CAMPO LIMPO
CANGAIBA
CAPAO REDONDO
CARRAO
CASA VERDE
CIDADE ADEMAR
CIDADE DUTRA
CIDADE LIDER
CIDADE TIRADENTES
CONSOLACAO
CURSINO
ERMELINO MATARAZZO
FREGUESIA DO O
GRAJAU
GUAIANASES
IGUATEMI
IPIRANGA
ITAIM BIBI
ITAIM PAULISTA
ITAQUERA
JABAQUARA
JACANA
MANDAQUI
MARSILAC
MOEMA
MOOCA
MORUMBI
PARI
PARQUE DO CARMO
PEDREIRA
PENHA
PERDIZES
PERUS
PINHEIROS
PIRITUBA
PONTE RASA
RAPOSO TAVARES
RIO PEQUENO
SACOMA
SANTANA
SANTO AMARO
SAO DOMINGOS
SAO LUCAS
SAO MATEUS
SAO MIGUEL
SAO RAFAEL
SAPOPEMBA
SAUDE
SE
SOCORRO
TATUAPE
TREMEMBE
TUCURUVI
VILA ANDRADE
VILA CURUCA
VILA FORMOSA
VILA GUILHERME
VILA JACUI
VILA LEOPOLDINA
VILA MARIA
VILA MARIANA
VILA MATILD